# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments               Customer  \
0  Mar 17 2023  8:30AM  259469        10     37888    Digital Brands, LLC   
1  Mar 17 2023  8:30AM  259469        10     37889    Digital Brands, LLC   
2  Mar 17 2023  8:30AM  259469        10     37890    Digital Brands, LLC   
3  Mar 17 2023  8:30AM  259469        10     37891    Digital Brands, LLC   
4  Mar 16 2023  3:37PM  259473        15  30014824  Alliance Pharma, Inc.   
5  Mar 16 2023  3:37PM  259473        15  30014825  Alliance Pharma, Inc.   
6  Mar 16 2023  3:37PM  259473        15  30014826  Alliance Pharma, Inc.   
7  Mar 16 2023  3:37PM  259473        15  30014846  Alliance Pharma, Inc.   
8  Mar 16 2023  3:37PM  259473        15  30014847  Alliance Pharma, Inc.   
9  Mar 16 2023  3:37PM  259473        15  30014838  Alliance Pharma, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
8   259469       Released          4
9   259470       Released          2
10  259471       Released          1
11  259472       Released          1
12  259473       Released         23

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
259469                NaN       4.0
259470                NaN       2.0
259471                NaN       1.0
259472                NaN       1.0
259473                NaN      23.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
259450                0.0       6.0
259451                0.0       1.0
259453                4.0       5.0
259460                0.0       1.0
259461                0.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
259450                  0         6
259451                  0         1
259453                  4         5
259460                  0         1
259461                  0         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               259450          0         6
1               259451          0         1
2               259453          4         5
3               259460          0         1
4               259461          0         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               259450                  6
1               259451                  1
2               259453         4        5
3               259460                  1
4               259461                  3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                     Customer
0   Mar 17 2023  8:30AM  259469        10          Digital Brands, LLC
4   Mar 16 2023  3:37PM  259473        15        Alliance Pharma, Inc.
27  Mar 16 2023  3:32PM  259472        19      GUSA Granules USA, Inc.
28  Mar 16 2023  3:26PM  259471        19        Eli Lilly and Company
29  Mar 16 2023  3:24PM  259470        20               Exact-Rx, Inc.
31  Mar 16 2023  2:44PM  259465        19       NAPP Technologies, LLC
32  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC
33  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.
42  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC
43  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                     Customer  \
0  Mar 17 2023  8:30AM  259469        10          Digital Brands, LLC   
1  Mar 16 2023  3:37PM  259473        15        Alliance Pharma, Inc.   
2  Mar 16 2023  3:32PM  259472        19      GUSA Granules USA, Inc.   
3  Mar 16 2023  3:26PM  259471        19        Eli Lilly and Company   
4  Mar 16 2023  3:24PM  259470        20               Exact-Rx, Inc.   
5  Mar 16 2023  2:44PM  259465        19       NAPP Technologies, LLC   
6  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
7  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
8  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
9  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   

  Executing Released  
0                  4  
1                 23  
2                  1  
3                  1  
4                  2  
5                  1  
6         1           
7         4        5  
8                  1  
9                  3

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                     Customer  \
0  Mar 17 2023  8:30AM  259469        10          Digital Brands, LLC   
1  Mar 16 2023  3:37PM  259473        15        Alliance Pharma, Inc.   
2  Mar 16 2023  3:32PM  259472        19      GUSA Granules USA, Inc.   
3  Mar 16 2023  3:26PM  259471        19        Eli Lilly and Company   
4  Mar 16 2023  3:24PM  259470        20               Exact-Rx, Inc.   
5  Mar 16 2023  2:44PM  259465        19       NAPP Technologies, LLC   
6  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
7  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
8  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
9  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   

  Released Executing  
0        4            
1       23            
2        1            
3        1            
4        2            
5        1            
6                  1  
7        5         4  
8        1            
9        3

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                 Customer Released  \
0  Mar 17 2023  8:30AM  259469        10      Digital Brands, LLC        4   
1  Mar 16 2023  3:37PM  259473        15    Alliance Pharma, Inc.       23   
2  Mar 16 2023  3:32PM  259472        19  GUSA Granules USA, Inc.        1   
3  Mar 16 2023  3:26PM  259471        19    Eli Lilly and Company        1   
4  Mar 16 2023  3:24PM  259470        20           Exact-Rx, Inc.        2   

  Executing  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                 Customer  Released  \
0  Mar 17 2023  8:30AM  259469        10      Digital Brands, LLC       4.0   
1  Mar 16 2023  3:37PM  259473        15    Alliance Pharma, Inc.      23.0   
2  Mar 16 2023  3:32PM  259472        19  GUSA Granules USA, Inc.       1.0   
3  Mar 16 2023  3:26PM  259471        19    Eli Lilly and Company       1.0   
4  Mar 16 2023  3:24PM  259470        20           Exact-Rx, Inc.       2.0   

   Executing  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                 Customer  Released  \
0  Mar 17 2023  8:30AM  259469        10      Digital Brands, LLC       4.0   
1  Mar 16 2023  3:37PM  259473        15    Alliance Pharma, Inc.      23.0   
2  Mar 16 2023  3:32PM  259472        19  GUSA Granules USA, Inc.       1.0   
3  Mar 16 2023  3:26PM  259471        19    Eli Lilly and Company       1.0   
4  Mar 16 2023  3:24PM  259470        20           Exact-Rx, Inc.       2.0   

   Executing  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10          259469       4.0        0.0
15         1037847      32.0        4.0
18          259463       0.0        1.0
19         1297309      10.0        0.0
20          259470       2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10   259469       4.0        0.0
1        15  1037847      32.0        4.0
2        18   259463       0.0        1.0
3        19  1297309      10.0        0.0
4        20   259470       2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10       4.0        0.0
1        15      32.0        4.0
2        18       0.0        1.0
3        19      10.0        0.0
4        20       2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released    4.0
1        15  Released   32.0
2        18  Released    0.0
3        19  Released   10.0
4        20  Released    2.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse   10    15   18    19   20
Status                              
Executing  0.0   4.0  1.0   0.0  0.0
Released   4.0  32.0  0.0  10.0  2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status   10    15   18    19   20
0          Executing  0.0   4.0  1.0   0.0  0.0
1           Released  4.0  32.0  0.0  10.0  2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status   10    15   18    19   20
0  Executing  0.0   4.0  1.0   0.0  0.0
1   Released  4.0  32.0  0.0  10.0  2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()